Zhang Chao, 2025/04/03<br>
Contribution of environmental factors to the cooling trends<br>

In [7]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import pandas as pd
import numpy as np
import xarray as xr
import geopandas as gpd
import seaborn as sns
import pymannkendall as mk
import statsmodels.api as sm
import regionmask
import matplotlib.lines as mlines
from scipy.stats import linregress
import sys
import matplotlib as mpl
import os
import scipy
import pingouin as pg
import cartopy.io.shapereader as shpreader
import matplotlib.colors as mcolors
sys.path.append('/home/climate/chaoz/code/utils/')
from plot_utils import plot_settings

In [8]:
os.chdir('/home/climate/chaoz/project/03Irr_Ts_CN/processed/')
BGC = xr.open_dataset('bgc_terraclimate_Yr_CN_1km_2001_2020.nc')
dLSTd = xr.open_dataset('delta_LSTday_Yr_CN_2001_2020.nc')
dLSTn = xr.open_dataset('delta_LSTnight_Yr_CN_2001_2020.nc')
dEVI  = xr.open_dataset('delta_EVI_Yr_CN_2001_2020.nc')

shp_cn = gpd.read_file('../shapefile/ChinaAll.shp')
shp_nanhai = gpd.read_file('../shapefile/Nanhai.shp')
shp_climzone = gpd.read_file('../shapefile/ClimateZone_3.shp')

In [9]:
def get_pcorrelation(dlst,iwu,pr,ta,sr,ws,devi,region):
    df = pd.DataFrame({
                     'dLst':dlst[region][2:].to_list(),
                     'dIwu':iwu[region].to_list(),
                     'Pr':pr[region][2:].to_list(),
                     'Ta':ta[region][2:].to_list(),
                     'Sr':sr[region][2:].to_list(),
                     'Ws':ws[region][2:].to_list(),
                     'dEvi':devi[region][2:].to_list(),
                     })

    slopes = {col: linregress(range(len(df[col])), df[col]).slope for col in df.columns}
    
    vars_y = ['dLst']
    vars_x = ['dIwu','Pr','Ta','Sr','Ws','dEvi']
    
    # Store results
    results = []
    
    for var in vars_x:
        covariates = [v for v in vars_x if v != var]  # Control for other variables
        result = pg.partial_corr(data=df, x=var, y=vars_y[0], covar=covariates, method='pearson')
        results.append({
            'Variable': var,
            'Partial_corr': result['r'].values[0],
            'P_value': result['p-val'].values[0]
        })
    
    # Convert to a nice DataFrame
    partial_corr_df = pd.DataFrame(results)
    
    return partial_corr_df

In [10]:
df_dLSTday   = pd.read_csv('regionmean_dLSTday_Yr.csv')
df_dLSTnight = pd.read_csv('regionmean_dLSTnight_Yr.csv')
df_IWU       = pd.read_csv('regionmean_IWU_Yr.csv')
df_pr        = pd.read_csv('regionmean_pr.csv')
df_sr        = pd.read_csv('regionmean_sr.csv')
df_ta        = pd.read_csv('regionmean_ta.csv')
df_ws        = pd.read_csv('regionmean_ws.csv')
df_dEVI      = pd.read_csv('regionmean_dEVI.csv')

In [11]:
'''
Get the regional trend contributed by iwu, background climate, and evi
'''
regions=['China','Arid','Semi','Humid']
df_con_day = [get_pcorrelation(df_dLSTday,df_IWU,df_pr,df_ta,df_sr,df_ws,df_dEVI,region) for region in regions]

df_con_night = [get_pcorrelation(df_dLSTnight,df_IWU,df_pr,df_ta,df_sr,df_ws,df_dEVI,region) for region in regions]


In [ ]:
p1,p2=0.01,0.05
yticklabels = ['IWU','Precipitation','Temperature', 'Solar radiation','Wind speed','$\Delta$EVI']#,'Res'


'''significance level'''
def getSigFlag(p):
    strSig = ''
    if p<p1:
        strSig = '**'
    elif p<p2:
        strSig = '*'
    else:
        strSig = ''
    return strSig


'''Design the bar color according the p value'''
def getbarcolor(p):
    p1,p2=0.01,0.05
    if p < p1:
        c = 'b'
    elif p< p2:
        c = '#4169E1'
    else:
        c = 'gray'
    return c

def DrawGraph(fig,pos,df,No,ylabelflag,xlabelflag,ylabel):
    ax = fig.add_axes(pos)
    pcor = df.iloc[:,1]
    pval   = df.iloc[:,2]

    yax = [6,5,4,3,2,1] # These are the y-coordinates for the center of your bars
    bars = ax.barh(y = yax,width=pcor,height=0.8,color=[getbarcolor(p) for p in pval])

    ax.spines['top'].set_visible(False)
    ax.spines['right'].set_visible(False)
    ax.set_xlim([-1.1,1.1])
    # ax.set_xticks([-0.25,0,0.25],[-0.25,0,0.25])

    ax.text(-0.11,1.02,No,weight='bold',fontsize=14,transform = ax.transAxes)
    if ylabelflag == True:
        # Ensure 'yticks' variable is defined if you use it here for labels
        # For example: yticks = ['Label1', 'Label2', ...]
        ax.set_yticks(yax, yticklabels) # Assuming the first column of df contains y-tick labels
        ax.set_ylabel(ylabel,weight='bold')
    else:
        ax.set_yticks(yax,['','','','','',''])
    if xlabelflag == True:
        ax.set_xlabel('$Partial correlation coefficient',y=-0.1) # y is deprecated, use labelpad

    ax.axvline(x=0.0, c="k", alpha=0.4, ls="--", lw=1)

    # --- Add significance symbols ---
    for i, (correlation, p_value) in enumerate(zip(pcor, pval)):
        symbol = getSigFlag(p_value)
        if symbol: # Only add text if there's a symbol
            bar_y_position = yax[i]-0.05
            bar_x_end = correlation

            # Adjust x_offset based on whether the bar is positive or negative
            x_offset = 0.02 # Small offset from the end of the bar
            text_x_position = bar_x_end + x_offset if bar_x_end >= 0 else bar_x_end - x_offset - 0.02 # adjust for symbol width if negative

            horizontal_alignment = 'left' if bar_x_end >= 0 else 'right'

            ax.text(text_x_position,
                    bar_y_position,
                    symbol,
                    va='center', # Vertically align to the center of the bar
                    ha=horizontal_alignment,
                    color='black', # Or choose another color
                    fontsize=12) # Adjust fontsize as needed
    
    return ax


colors = ['gray','#1E88E5','#33A02C','#FFA726']

fig = plt.figure(figsize=(9,5))
plot_settings()

x0,y0 = 0.15,0.09
xi,yi = 0.04,0.15
hx1,vx1 = (1-x0-3*xi-0.01)/4,0.35

pos11 = [x0+(hx1+xi)*0, y0+(vx1+yi)*1, hx1, vx1]
pos12 = [x0+(hx1+xi)*1, y0+(vx1+yi)*1, hx1, vx1]
pos13 = [x0+(hx1+xi)*2, y0+(vx1+yi)*1, hx1, vx1]
pos14 = [x0+(hx1+xi)*3, y0+(vx1+yi)*1, hx1, vx1]

pos21 = [x0+(hx1+xi)*0, y0+(vx1+yi)*0, hx1, vx1]
pos22 = [x0+(hx1+xi)*1, y0+(vx1+yi)*0, hx1, vx1]
pos23 = [x0+(hx1+xi)*2, y0+(vx1+yi)*0, hx1, vx1]
pos24 = [x0+(hx1+xi)*3, y0+(vx1+yi)*0, hx1, vx1]

ax11 = DrawGraph(fig, pos11,df_con_day[0]   ,'a',True , False, 'Day')
ax12 = DrawGraph(fig, pos12,df_con_day[1]   ,'b',False, False, '')
ax13 = DrawGraph(fig, pos13,df_con_day[2]   ,'c',False, False, '')
ax14 = DrawGraph(fig, pos14,df_con_day[3]   ,'d',False, False, '')
ax21 = DrawGraph(fig, pos21,df_con_night[0] ,'e',True , False, 'Night')
ax22 = DrawGraph(fig, pos22,df_con_night[1] ,'f',False, False, '')
ax23 = DrawGraph(fig, pos23,df_con_night[2] ,'g',False, False, '')
ax24 = DrawGraph(fig, pos24,df_con_night[3] ,'h',False, False, '')

ax11.set_title('China')
ax12.set_title('Arid')
ax13.set_title('Semi-arid/humid',loc='right')
ax14.set_title('Humid')

# ax21.set_xlabel('',loc='right')
ax22xl = ax22.set_xlabel('Partial Correlation Coefficient')
ax22xl.set_position((1.2,0))


plt.savefig('../figures/Figure_S09.png',dpi=300)